In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import time
import os

from bs4 import BeautifulSoup
import re
import pandas as pd

import requests
import os
import urllib
from urllib.parse import urlparse

import subprocess

In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException

In [ ]:
from tools import sync_get_element_by_xpath, sync_get_elements_by_xpath, take_screenshot, close_chat_panel, sign_in

In [ ]:
#settings

WEB_DRIVER_DEALY = 10

In [ ]:
# default settings for this Udacity program

download_root_folder = ""
current_read_write_path = ""

# hard-coded entry url for the program
root_address = 'https://learn.udacity.com/nanodegrees/nd880/parts/cd0569'

In [ ]:
#setup option for chrome profile
chrome_options = Options()

chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
## the following option is to solve the error if using the above cause a problem
# chrome_options.add_argument("--remote-debugging-port=9222") 

prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
chrome_options.add_experimental_option("prefs", prefs)

#start web driver
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(WEB_DRIVER_DEALY)
wait = WebDriverWait(driver, WEB_DRIVER_DEALY)

In [ ]:
# go to udacity
driver.get(root_address)

In [ ]:
# sign in
sign_in(driver,wait)

In [ ]:
# get the name of the program and create the root folder for storing the scraped result
program_name = sync_get_element_by_xpath(driver, wait, "//h1[@class='chakra-heading css-10cnqhb']").get_attribute("innerHTML")
download_root_folder = program_name
current_read_write_path = download_root_folder
os.makedirs(current_read_write_path, exist_ok=False)

In [ ]:
# get the two main part of this program. Hard coded!
course_content_blk1 = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tab-panels css-8atqhb']/div[1]//div[@class='css-pk1kta']")[0]
course_content_blk2 = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tab-panels css-8atqhb']/div[1]//div[@class='css-1vc4pt9']")[0] 

In [ ]:
# create folder structure for storing the scraped data for the Udacity program
title_first_part = course_content_blk1.find_element("xpath", "./div/div/button/div/div[1]/div/div/h2").get_attribute("innerHTML")
title_second_part = course_content_blk2.find_element("xpath", "./div/div/button/div/div[1]/div/div/h2").get_attribute("innerHTML")

# keep reminding myself the current read/write folder
current_target_folder = current_read_write_path+"/1. "+title_first_part
os.makedirs(current_target_folder, exist_ok=False)
current_target_folder = current_read_write_path+"/2. "+title_second_part
os.makedirs(current_target_folder, exist_ok=False)

In [ ]:
# get all the web elements of topics in the first part of the program
class_as = course_content_blk1.find_elements("xpath", "./div/div/div/div/a")

In [ ]:
current_read_write_path = download_root_folder+"/1. "+title_first_part
topic_urls = []
topic_as = course_content_blk1.find_elements("xpath", "./div/div/div/div/a")

# for idx in range(len(topic_as)):
for idx in[0]:
    topic_a = topic_as[idx]
    topic_title = topic_a.find_element("xpath","./div/h3").get_attribute("innerHTML")
    current_target_folder = current_read_write_path+"/{}. ".format(idx+1)+topic_title
    os.makedirs(current_target_folder, exist_ok=False)
    print("create folder: {}".format(current_target_folder))

    url = topic_a.get_attribute("href")
    driver.get(url)
    close_chat_panel(driver, wait)
    # loop though sub-topic
    lis = sync_get_elements_by_xpath(driver,wait, "//ul[@class='css-1kfpgwe']/li")
    for li in lis:
        # open right side pane for switch sub-topics
        toggle_open_btn = sync_get_element_by_xpath(driver,wait, "//button[@class='chakra-button css-wc1vc9']")
        if toggle_open_btn is not None:
            toggle_open_btn.click()
        
        # swich sub-topic
        li.click()
        
        # close right side pane for taking screenshots
        toggle_close_btn = sync_get_element_by_xpath(driver,wait, "//button[@class='chakra-button css-1rtwe76']")
        if toggle_close_btn is not None:
            toggle_close_btn.click()

        # create sub-topic folder 
        sub_topic_title = li.find_element("xpath","./a/p").get_attribute("innerHTML")
        current_target_folder = current_read_write_path+"/{}. ".format(idx+1)+topic_title+"/"+sub_topic_title
        os.makedirs(current_target_folder, exist_ok=False)
        print("create folder: {}".format(current_target_folder))

        time.sleep(3)
        take_screenshot(driver.current_url,current_target_folder+"/screenshot.png",WEB_DRIVER_DEALY)


    topic_urls.append(url)


In [ ]:
# toggle_open_btn = sync_get_element_by_xpath(driver,wait, "//button[@class='chakra-button css-wc1vc9']")
# if toggle_open_btn is not None:
#     toggle_open_btn.click()
# lis[1].click()
# time.sleep(3)
# toggle_close_btn = sync_get_element_by_xpath(driver,wait, "//button[@class='chakra-button css-1rtwe76']")
# if toggle_close_btn is not None:
#     toggle_close_btn.click()


In [ ]:
# lis = sync_get_elements_by_xpath(driver,wait, "//ul[@class='css-1kfpgwe']/li")
# lis[0].find_element("xpath","./a").get_attribute("href")


In [ ]:
# for topic_url in topic_urls:
#     print(topic_url)

In [ ]:
# driver.get(topic_urls[0])

In [ ]:
# # get tab button web elements for different types of Curriculum: Core, Elective and Career
# course_tab_elms = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tab-panel css-1qbr3jw']")

# tab_buttons = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tablist css-1cul31m']/button")
# for tab_button in tab_buttons:
#     print("Type: {}".format(tab_button.get_attribute("innerHTML")))


In [ ]:
# take_screenshot("https://learn.udacity.com/nanodegrees/nd880/parts/cd0569/lessons/288b2d68-a082-4bb4-96a3-7a353f662aa2/concepts/4feddc57-8a5f-48a8-82f0-b3d90b8f47aa","main.png",WEB_DRIVER_DEALY)
